
<hr/>
<h2 id="title:-Working-With-Numpy-Arrays">Working With Numpy Arrays<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#title:-Working-With-Numpy-Arrays"></a></h2>


Relevant VMTK scripts


<p>Scripts that convert VMTK objects to numpy arrays:</p>
<ol>
<li>vmtkcenterlinestonumpy </li>
<li>vmtkimagetonumpy</li>
<li>vmtksurfacetonumpy</li>
</ol>
<p>Scripts that convert a nested dictionary of numpy arrays to VMTK objects:</p>
<ol>
<li>vmtknumpytocenterlines</li>
<li>vmtknumpytoimage</li>
<li>vmtknumpytosurface</li>
</ol>
<p>Convenience scripts to write and read a nested dictionary of numpy arrays to disk:</p>
<ol>
<li>vmtknumpyreader </li>
<li>vmtknumpywriter </li>
</ol>
<hr/>



<p>In addition to the standard VMTK package, the following packages must be installed and available on the users PATH:</p>
<ol>
<li><a href="http://www.numpy.org/">numpy</a></li>
<li><a href="http://www.h5py.org/">h5py</a></li>
</ol>
<p>We recommend using the <a href="https://anaconda.org/">Python Anaconda</a> package manager to create a virtual environment and install the packages. Installation and quickstart instructions are available <a href="https://docs.continuum.io/docs_oss/conda/get-started">here</a>.</p>
<hr/>



<h2 id="Structured-Conversion-Between-VTK-Data-and-Numpy-Arrays">Structured Conversion Between VTK Data and Numpy Arrays<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#Structured-Conversion-Between-VTK-Data-and-Numpy-Arrays"></a></h2>



<p>When we want to access VTK data in numpy, we actually need to access the four principle component of a VTK object as described above:</p>
<ol>
<li>Points</li>
<li>Cells</li>
<li>Point Data</li>
<li>Cell Data</li>
</ol>
<p>In order to make use of the data in numpy/python, we create a unique numpy array for every VTK array defining these components. In order to handle these arrays in a manageable fashion (accessible through one python object), we assign each data array to a key/value pair in a (nested) python dictionary of pre-defined structure. The following subsections defines the dictionary structure required for centerlines, images, and surfaces.</p>
<p>Note: The following structure must be reproduced <i> exactly </i> in order to successfully convert a numpy dataset back into VMTK. Be sure to include all dictionary components (even if they are empty) and ensure that all keys exactly match the description below / the output from converting VMTK objects to numpy.</p>



<h3 id="Converting-VMTK-Surface-Objects">Converting VMTK Surface Objects<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#Converting-VMTK-Surface-Objects"></a></h3>



<p>The following example demonstrates how to convert a VMTK surface to a numpy array. This is handled by the <code>vmtkscripts.vmtkSurfaceToNumpy()</code> script which accepts a <code>Surface</code> as a input and outputs the nested dictionary of arrays through the <code>ArrayDict</code> class attribute.</p>



<p>The structure of the <code>ArrayDict</code> nested dictionary is as follows:</p>
<pre><code>ArrayDict
    ['Points']                   &lt;-- required, is Nx3 array of N vertexes and x, y, z locations
    ['PointData']                &lt;-- required, even if subarrays are empty
        ['PointDataArray1']      &lt;-- optional, (ex. MaximumInscribedSphereRadius)
        ['PointDataArray2']      &lt;-- optional
        ...
    ['CellData']                 &lt;-- required
        ['CellPointIds']         &lt;-- required, is Mx3 array defines cell conectivity to ['Points] Indices
        ['CellDataArray1']       &lt;-- optional, (ex. CenterlineTractId)
        ['CellDataArray2']       &lt;-- optional
        ...</code></pre>



<p>An example showing the dictionary structure is below:</p>


In [ ]:
from vmtk import vmtkscripts
import numpy as np

surfaceReader = vmtkscripts.vmtkSurfaceReader()
surfaceReader.InputFileName = '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case11/surfaces/surfaceFinal.vtp'
surfaceReader.Execute()

surfaceNumpyAdaptor = vmtkscripts.vmtkSurfaceToNumpy()
surfaceNumpyAdaptor.Surface = surfaceReader.Surface
surfaceNumpyAdaptor.Execute()

numpySurface = surfaceNumpyAdaptor.ArrayDict


<p>The python object <code>numpySurface</code> is now a nested dictionary of numpy arrays. The top level key/value pairs are:</p>


In [2]:
numpySurface

{'CellData': {'CellPointIds': array([[    0,     1,     2],

         [    3,     4,     5],

         [    6,     7,     8],

         ..., 

         [42457, 38681, 45910],

         [40090, 44301, 33428],

         [46260, 40090, 33428]], dtype=int32)},

 'PointData': {'ClippingArray': array([  70.2382128 ,   70.7417044 ,   68.85387999, ...,   37.22577181,

          171.85029787,   82.39789181]),

  'GroupIds': array([0, 0, 0, ..., 0, 0, 0], dtype=int32)},

 'Points': array([[ 52.02830124,  78.64040375,  63.9836998 ],

        [ 51.91070175,  78.67320251,  63.95230103],

        [ 51.96960068,  78.74160004,  64.04229736],

        ..., 

        [ 49.19287491,  83.13684845,  77.23335266],

        [ 49.60110092,  73.71900177,  84.4803009 ],

        [ 52.77119827,  75.76010132,  64.35810089]], dtype=float32)}

In [3]:
numpySurface

{}


<p>For <code>numpySurface</code>, we can see that the numpy of surface points (stored in <code>numpySurface['Points']</code>) and the number of triangles (stored in <code>numpySurface['CellData']['CellPointIds']</code>) is:</p>


In [ ]:
print('Number of Points = ', numpySurface['Points'].shape[0], ' Number of Dimensions = ', numpySurface['Points'].shape[1])
print('Number of Faces = ', numpySurface['CellData']['CellPointIds'].shape[0], ' Number of Vertices Per Face = ', numpySurface['CellData']['CellPointIds'].shape[1])


<p>We can also see that the data stored in <code>PointData</code> has the shape:</p>


In [ ]:
print('numpySurface["PointData"]["DistanceToCenterlines"] shape = ', numpySurface["PointData"]["DistanceToCenterlines"].shape)


<p>which exactly matches the number of vertices in the <code>Points</code> array. In this case, each index in <code>numpySurface['PointData']['DistanceToCenterlines']</code> corresponds to a row in <code>numpySurface['Points']</code>. ie. <code>numpySurface['PointData']['DistanceToCenterlines'][100]</code> corresponds to the vertex defined by coordinates at <code>numpySurface['Points'][100, :]</code></p>
<p>From here, we can modify the location of the points, define a new cell connectivity list, or add a new Point Data or Cell Data array. If we decide we want to modify the <code>DistanceToCenterlines</code> array values by a constant scale factor, we can perform the operation and convert back to a VMTK surface Object by:</p>


In [ ]:
print('Before Modification: ', numpySurface['PointData']['DistanceToCenterlines'])
numpySurface['PointData']['DistanceToCenterlines'] = numpySurface['PointData']['DistanceToCenterlines'] * 2
print('After Modification: ', numpySurface['PointData']['DistanceToCenterlines'])

surfaceVmtkAdaptor = vmtkscripts.vmtkNumpyToSurface()
surfaceVmtkAdaptor.ArrayDict = numpySurface
surfaceVmtkAdaptor.Execute()


<p>We can then use the VMTK object as normal or save it to a file.</p>



<h3 id="Converting-VMTK-Centerline-Objects">Converting VMTK Centerline Objects<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#Converting-VMTK-Centerline-Objects"></a></h3>



<p>The following example demonstrates how to convert a VMTK centerline to a numpy array. This is handled by the <code>vmtkscripts.vmtkCenterlinesToNumpy()</code> script which accepts a <code>Surface / Centerline</code> as a input and outputs the nested dictionary of arrays through the <code>ArrayDict</code> class attribute.</p>



<p>The structure of the <code>ArrayDict</code> nested dictionary is as follows:</p>
<pre><code>ArrayDict
    ['Points']                   &lt;-- required, is Nx3 array of N vertexes and x, y, z locations
    ['PointData']                &lt;-- required, even if subarrays are empty
        ['PointDataArray1']      &lt;-- optional, (ex. MaximumInscribedSphereRadius)
        ['PointDataArray2']      &lt;-- optional
        ...
    ['CellData']                 &lt;-- required
        ['CellPointIds']         &lt;-- required, list of Mx1 arrays defining cell connectivity to ['Points']
        ['CellDataArray1']       &lt;-- optional, (ex: CenterlineTractId)
        ['CellDataArray2']       &lt;-- optional
        ...</code></pre>



<p>Note: The format for <code>['CellData']['CellPointIds']</code> is slightly different for centerlines than it is for the surface example above. Unlike a surface (which defined each triangle face as a row in <code>['CellData']['CellPointIds']</code> with fixed dimensions Mx3), the number of Points making up each cell in a centerline are completely arbitrary. Instead of a <code>Mx3</code> size array, the centerline version of <code>['CellData']['CellPointIds']</code> contains a list of numpy arrays with (potentially) non-equal sizes. Aside from this semantic difference, the meaning of each index in <code>['CellData']['CellPointIds']</code> is the same for centerlines and surfaces; that is, each value in <code>['CellData']['CellPointIds']</code> refers to a row index in <code>['Points']</code> which contains the coordinates of the associated points.</p>


In [ ]:
centerlineReader = vmtkscripts.vmtkSurfaceReader()
centerlineReader.InputFileName = 'foo/coronary-test-centerline-branches.vtp'
centerlineReader.Execute()

clNumpyAdaptor = vmtkscripts.vmtkCenterlinesToNumpy()
clNumpyAdaptor.Centerlines = centerlineReader.Surface
clNumpyAdaptor.Execute()

numpyCenterlines = clNumpyAdaptor.ArrayDict


<p>Like the surface example above, we can see that the data accessed through <code>numpyCenterlines['Points']</code> is a numpy array of shape Nx3:</p>


In [ ]:
print('numpyCenterlines["Points"] shape = ', numpyCenterlines['Points'].shape)


<p>In addition, we see that the keys for Point Data and Cell Data are:</p>


In [ ]:
print('Point Data Keys: ', numpyCenterlines['PointData'].keys())
print('Cell Data Keys: ', numpyCenterlines['CellData'].keys())


<p>We see that the number of components in <code>['PointData']['MaximumInscribedSphereRadius']</code> is equal to the number of rows in <code>['Points']</code></p>


In [ ]:
print('Point Data Shape: ', numpyCenterlines['PointData']['MaximumInscribedSphereRadius'].shape, ' = Number of Points: ', numpyCenterlines['Points'].shape[0])


<p>We can also see that <code>['CellData']['CellPointIds']</code> is a list of a certain length which contains a series of numpy arrays, and that the sizes of the arrays are not necesarily the same:</p>


In [ ]:
print("['CellData']['CellPointIds] is a python list: ", isinstance(numpyCenterlines['CellData']['CellPointIds'], list))
print("length of ['CellData']['CellPointIds'] list = ", len(numpyCenterlines['CellData']['CellPointIds']), '\n')

minSize = np.inf
maxSize = 0
for index, cellId in enumerate(numpyCenterlines['CellData']['CellPointIds']):
    if cellId.shape[0] > maxSize:
        maxSize = cellId.shape[0]
        maxSizeCellIndex = index
    if cellId.shape[0] < minSize:
        minSize = cellId.shape[0]
        minSizeCellIndex = index
        
print("Minimum size of array in ['CellData']['CellPointIds'] = ", minSize, ' Which occured at index ', minSizeCellIndex)
print("Maximum size of array in ['CellData']['CellPointIds'] = ", maxSize, ' Which occured at index ', maxSizeCellIndex)


<p>We can the see that the data inside of <code>['CellData']</code> (not at <code>['CellPointIds']</code>) has a shape equal to the length of the <code>['CellData']['CellPointIds']</code>  list</p>


In [ ]:
for cellDataKey in numpyCenterlines['CellData']:
    if cellDataKey == 'CellPointIds':
        pass
    else:
        print('Shape of ', cellDataKey, ' = ', numpyCenterlines['CellData'][cellDataKey].shape)


<p>This is important, as the data stored in the <code>['CellData']</code> arrays are indexed so that the value at <code>['CellData']['foo'][index]</code> refers to the list element at the corresponding list index in <code>['CellData']['CellPointIds']</code>. For example: the data sorted at <code>['CellData']['TractIds'][5]</code> refers to the cell point id list accessible at <code>['CellData']['CellPointIds'][5]</code>. Recall that this array's values indicate the row indices of the vertices in <code>['Points']</code> which make up the cell. In this way it is possible to map a Cell Data value to a particular Point coordinate.</p>
<p>However, it is not always convenient to have to map each <code>['CellData']['foo']</code> index to the <code>['CellData']['CellPointIds']</code> list to the corresponding coordinate in <code>['Points']</code>. A conceptually simpler (though more resource intensive) method to perform the mapping is to take each dataset attribute in <code>['CellData']</code> (with the exception of <code>CellPointIDs</code>), and map it to a dense array in <code>['PointData']</code>. This can be performed automatically by specifying <code>foo.ConvertCellToPoint = 1</code> when calling <code>vmtkscripts.vmtkCenterlinesToNumpy()</code></p>
<p>Like the surface example above, it is possible to convert from the nested python dictionary of arrays to a VMTK centerlines object via the <code>vmtkscripts.vmtkNumpyToCenterlines()</code> module:</p>


In [ ]:
clVmtkAdaptor = vmtkscripts.vmtkNumpyToCenterlines()
clVmtkAdaptor.ArrayDict = numpyCenterlines
clVmtkAdaptor.Execute()


<h3 id="Converting-VMTK-Image-Objects">Converting VMTK Image Objects<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#Converting-VMTK-Image-Objects"></a></h3>



<p>The conversion of VMTK Image Objects requires a much simpler ArrayDict structure than for a surface or centerline. The structure of the <code>ArrayDict</code> is as follows:</p>
<pre><code>ArrayDict
    ['Origin']                 &lt;-- required, is 3x1 array of (float) x,y,z vertex locations at index (0,0,0)
    ['Dimensions']             &lt;-- required, is 3x1 array of (int) number of voxels in the x,y,z direction.
    ['Spacing']                &lt;-- required, is 3x1 array of (float) voxel spacing in the x,y,z directions
    ['PointData']              &lt;-- required
        ['PointDataArray1']    &lt;-- required, is np.ndarray of shape = dimensions containing image data. 
        ['PointDataArray2']    &lt;-- optional, is np.ndarray of shape = dimensions containing image data. 
        ...</code></pre>



<p>Recall that for this module, image data does not have a concept of cell data. As a result the entire VTK object can be specified with 3 attributes (Origin, Dimensions, &amp; Spacing) and a dense array representing the image intensity scalars at each voxel index. An example of this is shown below:</p>


In [ ]:
imageReader = vmtkscripts.vmtkImageReader()
imageReader.InputFileName = 'foo/aorta-voi.mha'
imageReader.Execute()

imageNumpyAdaptor = vmtkscripts.vmtkImageToNumpy()
imageNumpyAdaptor.Image = imageReader.Image
imageNumpyAdaptor.Execute()

numpyImage = imageNumpyAdaptor.ArrayDict

In [ ]:
print("['Spacing'] = ", numpyImage['Spacing'])
print("['Origin'] = ", numpyImage['Origin'])
print("['Dimensions'] = ", numpyImage['Dimensions'])
print("['PointData']['ImageScalars'] shape = ", numpyImage['PointData']['ImageScalars'].shape)


<p>We can modify the data stored in <code>['PointData']['ImageScalars']</code> and return it to a VMTK image object as demonstrated below:</p>


In [ ]:
print('Intensity Maximum Before Modification = ', np.amax(numpyImage['PointData']['ImageScalars']))
print('Intensity Minimum Before Modification = ', np.amin(numpyImage['PointData']['ImageScalars']))

numpyImage['PointData']['ImageScalars'][np.where(numpyImage['PointData']['ImageScalars'] > 500)] = 500

print('Intensity Maximum After Modification = ', np.amax(numpyImage['PointData']['ImageScalars']))
print('Intensity Minimum After Modification = ', np.amin(numpyImage['PointData']['ImageScalars']))

In [ ]:
imageVmtkAdaptor = vmtkscripts.vmtkNumpyToImage()
imageVmtkAdaptor.ArrayDict = numpyImage
imageVmtkAdaptor.Execute()


<h2 id="Saving-VMTK-Numpy-Objects">Saving VMTK Numpy Objects<a class="anchor-link" href="http://www.vmtk.org/tutorials/WorkingWithNumpyArraysIPYNB.html#Saving-VMTK-Numpy-Objects"></a></h2>



<p>Two convenience scripts have been provided to save and read VMTK numpy objects (surfaces, centerlines, or images) to disk. The files can be written in either HDF5 format (provided the h5py module is installed) or via a standard python pickle object. The procedure is identical for every object type.</p>
<p>Simply use vmtkscripts.vmtkNumpyWriter() and vmtkscripts.vmtkNumpyReader() in the fashion standard through vmtk. The scripts take an <code>ArrayDict</code> as an input member along with a <code>Format</code> identification (default = pickle) and a output/input file name, and read/write the array to/from disk.</p>
